In [ ]:
from astropy.io import fits
from astropy.time import Time
import numpy as np
import exoplanet as xo
import matplotlib.pyplot as pl
import mpl_defaults as mpld
%matplotlib inline

In [ ]:
# source: https://sohowww.nascom.nasa.gov/data/archive.html
blue = fits.open('blue.fits')
green = fits.open('green.fits')
red = fits.open('red.fits')

r, g, b = red[0].data, green[0].data, blue[0].data
mask = np.isfinite(r) & np.isfinite(g) & np.isfinite(b)
start = blue[0].header['DATES'][0:9]
end = blue[0].header['DATES'][14:]
start, end = Time([start, end]).jd
t = np.linspace(start, end, len(r)) - start
r, g, b = r[mask].astype('float64'), g[mask].astype('float64'), b[mask].astype('float64')
t = t[mask]
flux = np.sum([r, g, b], axis=0)/3

In [ ]:
flux = flux/1e3
f = np.fft.rfftfreq(len(t), t[1] - t[0])
fft = np.fft.rfft(flux)
fft *= np.conj(fft)
power = fft.real / len(t)**2

In [ ]:
fig, ax = pl.subplots(2, 1, figsize=(10, 14))
ax[0].loglog(f, power, '.k', ms=1)
ax[0].set_xlabel('frequency')
ax[0].set_ylabel(r'power (ppt$^2$/Hz)')

ax[1].loglog(f, power, '.k', ms=1)
ax[1].set_xlim(200, 400)
ax[1].set_ylabel(r'power (ppt$^2$/Hz)')
ax[1].set_xlabel('frequency')
ax[1].set_ylim(1e-13, 1e-8)

In [ ]:
i = 160000
di = 2000
t = t[i:i+di]
yerr = np.exp(-10)*np.ones_like(t)
r, g, b, flux = r[i:i+di]/1e3, g[i:i+di]/1e3, b[i:i+di]/1e3, flux[i:i+di]/1e3

In [ ]:
fig, ax = pl.subplots(4, 1, figsize=(10, 7), sharex=True)
ax[0].plot(t, 1+r, '.', color=mpld.red, alpha=1, ms=3.0)
ax[1].plot(t, 1+g, '.', color=mpld.green, alpha=1, ms=3.0)
ax[2].plot(t, 1+b, '.', color=mpld.blue, alpha=1, ms=3.0)
ax[3].plot(t, 1+flux, '.', color='k', alpha=1, ms=3.0)

ax[0].annotate('862 nm', xy=(0.85, 0.78), xycoords='axes fraction', fontsize=15)
ax[1].annotate('500 nm', xy=(0.85, 0.78), xycoords='axes fraction', fontsize=15)
ax[2].annotate('402 nm', xy=(0.85, 0.78), xycoords='axes fraction', fontsize=15)
ax[3].annotate('total flux', xy=(0.85, 0.78), xycoords='axes fraction', fontsize=15)

[ax.set_ylim(0.5, 1.5) for ax in ax]
ax[3].set_xlabel('days since Jan. 23, 1996')
pl.annotate("normalized flux", xy=(0.0, 0.75), 
            xycoords='figure fraction', 
            rotation=90, fontsize=30)
pl.savefig("/Users/tgordon/Desktop/spm_lc.pdf")

In [ ]:
f = np.fft.rfftfreq(len(t), t[1] - t[0])
fftr = np.fft.rfft(r)
fftg = np.fft.rfft(g)
fftb = np.fft.rfft(b)

fftr *= np.conj(fftr)
fftg *= np.conj(fftg)
fftb *= np.conj(fftb)

powerfftr = fftr.real / len(t)**2
powerfftg = fftg.real / len(t)**2
powerfftb = fftb.real / len(t)**2

In [ ]:
fig, ax = pl.subplots(3, 1, figsize=(10, 7), sharex=True)
ax[0].loglog(f, powerfftr, '.', color=mpld.red)
ax[1].loglog(f, powerfftg, '.', color=mpld.green)
ax[2].loglog(f, powerfftb, '.', color=mpld.blue)
pl.xlabel('frequency')

In [ ]:
log_S0 = [-9, -12, -15]
log_S0_bounds = [[-11, 0], [-14, -11], [-20, -14]]
log_w0 = [1, 6, 7.4]
log_Q = [-np.log(np.sqrt(2)), -np.log(np.sqrt(2)), 1]
log_sig = -18

import pymc3 as pm
import theano.tensor as tt

def find_map_soln(y):
    with pm.Model() as model:

        # The parameters of the SHOTerm kernel
        logS = [pm.Uniform("logS{0}".format(i), 
                        lower=log_S0_bounds[i][0], upper=log_S0_bounds[i][1], 
                        testval=ls) for i, ls in enumerate(log_S0)]
        logQ = [pm.Uniform("logQ{0}".format(i), 
                       lower=-20.0, upper=20.0, 
                       testval=lq) for i, lq in enumerate(log_Q)]
        logw = [pm.Uniform("logw{0}".format(i), 
                        lower=-20.0, upper=20.0, 
                        testval=lw) for i, lw in enumerate(log_w0)]
        mean = pm.Uniform("mean", lower=-1000, upper=1000, testval=0)
        #logsig = pm.Uniform("logsig", lower=-50, upper=0, testval=log_sig)
        kernel = xo.gp.terms.Term()
        for (ls, lw, lq) in zip(logS, logw, logQ):
            kernel += xo.gp.terms.SHOTerm(
            log_S0 = ls,
            log_w0 = lw,
            log_Q=lq)
    
        gp = xo.gp.GP(kernel, t, yerr ** 2, J=2*len(log_S0))
        pm.Potential("loglike", gp.log_likelihood(y-mean))
        
        map_soln = xo.optimize(start=model.test_point, verbose=False)
        power = xo.eval_in_model(gp.kernel.psd(2*np.pi*f), map_soln)
        #trace = pm.sample(
        #    tune=500,
        #    draws=2000,
        #    start=map_soln,
        #    cores=2,
        #    chains=2,
        #    step=xo.get_dense_nuts_step(target_accept=0.9)
        #)
      
    return model, map_soln#, power#, trace

In [ ]:
model_r, soln_r = find_map_soln(r)
model_g, soln_g = find_map_soln(g)
model_b, soln_b = find_map_soln(b)
#model_all, soln_all, power_all, trace_all = find_map_soln(flux)

In [ ]:
soln_b

In [ ]:
terms_r = []
for i in range(len(log_S0)):
    terms_r.append(xo.gp.terms.SHOTerm(
        log_S0 = soln_r['logS{0}'.format(i)],
        log_w0 = soln_r['logw{0}'.format(i)],
        log_Q = soln_r['logQ{0}'.format(i)]
    ).psd(2*np.pi*f).eval())
terms_g = []
for i in range(len(log_S0)):
    terms_g.append(xo.gp.terms.SHOTerm(
        log_S0 = soln_g['logS{0}'.format(i)],
        log_w0 = soln_g['logw{0}'.format(i)],
        log_Q = soln_g['logQ{0}'.format(i)]
    ).psd(2*np.pi*f).eval())
terms_b = []
for i in range(len(log_S0)):
    terms_b.append(xo.gp.terms.SHOTerm(
        log_S0 = soln_b['logS{0}'.format(i)],
        log_w0 = soln_b['logw{0}'.format(i)],
        log_Q = soln_b['logQ{0}'.format(i)]
    ).psd(2*np.pi*f).eval())

In [ ]:
fig, ax = pl.subplots(3, 1, figsize=(10, 15))
ax[0].loglog(f, powerfftr, '.', color=mpld.red)
#ax[0].loglog(f, power_r, '-k',  linewidth=2)
ax[0].loglog(f, terms_r[0], color='k', linewidth=1)
ax[0].loglog(f, terms_r[1], color='k', linewidth=1)
ax[0].loglog(f, terms_r[2], color='k', linewidth=1)

ax[1].loglog(f, powerfftg, '.', color=mpld.green)
#ax[1].loglog(f, power_g, '-k', linewidth=2)
ax[1].loglog(f, terms_g[0], color='k', linewidth=1)
ax[1].loglog(f, terms_g[1], color='k', linewidth=1)
ax[1].loglog(f, terms_g[2], color='k', linewidth=1)

ax[2].loglog(f, powerfftb, '.', color=mpld.blue)
#ax[2].loglog(f, power_b, '-k', linewidth=2)
ax[2].loglog(f, terms_b[0], color='k', linewidth=1)
ax[2].loglog(f, terms_b[1], color='k', linewidth=1)
ax[2].loglog(f, terms_b[2], color='k', linewidth=1)

[ax.set_xlim(1, 700) for ax in ax]
ax[0].set_ylim(1e-10, 1e-4)
ax[1].set_ylim(1e-10, 1e-4)
ax[2].set_ylim(1e-10, 1e-3)
pl.xlabel('frequency')
[ax.set_ylabel('power (ppt)') for ax in ax]
pl.savefig('/Users/tgordon/Desktop/spm_fits.pdf')

In [ ]:
import corner
samples = pm.trace_to_dataframe(trace_r)
_ = corner.corner(samples)
samples = pm.trace_to_dataframe(trace_g)
_ = corner.corner(samples)
samples = pm.trace_to_dataframe(trace_b)
_ = corner.corner(samples)

In [ ]:
pl.figure(figsize=(10, 7))
pl.hist(trace_b.get_values('logS2') - trace_g.get_values('logS2'), 
        bins=20, density=True, alpha=0.8, color=mpld.red, label="third component")
pl.hist(trace_b.get_values('logS1') - trace_g.get_values('logS1'), 
        bins=20, density=True, alpha=0.8, color=mpld.blue, label="second component");
pl.ylabel(r'~ probability')
pl.xlabel(r'$\log (S_\mathrm{blue}/S_\mathrm{green})$')
pl.legend()
pl.ylim(0, 2)
pl.savefig('/Users/tgordon/Desktop/prob_bg.pdf')

In [ ]:
pl.figure(figsize=(10, 7))
pl.hist(trace_g.get_values('logS2') - trace_r.get_values('logS2'), 
        bins=20, density=True, alpha=0.8, color=mpld.red, label="third component")
pl.hist(trace_g.get_values('logS1') - trace_r.get_values('logS1'), 
        bins=20, density=True, alpha=0.8, color=mpld.blue, label="second component");
pl.ylabel(r'~ probability')
pl.xlabel(r'$\log (S_\mathrm{green}/S_\mathrm{red})$')
pl.legend()
pl.ylim(0, 2)
pl.savefig('/Users/tgordon/Desktop/prob_gr.pdf')

In [ ]:
pl.figure(figsize=(10, 7))
pl.hist(trace_b.get_values('logS2') - trace_r.get_values('logS2'), 
        bins=20, density=True, alpha=0.8, color=mpld.red, label="third component")
pl.hist(trace_b.get_values('logS1') - trace_r.get_values('logS1'), 
        bins=20, density=True, alpha=0.8, color=mpld.blue, label="second component");
pl.ylabel(r'~ probability')
pl.xlabel(r'$\log (S_\mathrm{blue}/S_\mathrm{red})$')
pl.legend()
pl.ylim(0, 2)
pl.savefig('/Users/tgordon/Desktop/prob_br.pdf')

In [ ]:
pl.figure(figsize=(10, 7))
pl.hist(trace_g.get_values('logw2') - trace_r.get_values('logw2'), 
        bins=20, density=True, alpha=0.8, color=mpld.red, label="third component")
pl.hist(trace_g.get_values('logw1') - trace_r.get_values('logw1'), 
        bins=20, density=True, alpha=0.8, color=mpld.blue, label="second component");
pl.ylabel(r'~ probability')
pl.xlabel(r'$\log (\omega_\mathrm{green}/\omega_\mathrm{red})$')
pl.legend()
pl.savefig('/Users/tgordon/Desktop/prob_omega_gr.pdf')
#pl.ylim(0, 2)

In [ ]:
pl.figure(figsize=(10, 7))
pl.hist(trace_b.get_values('logw2') - trace_r.get_values('logw2'), 
        bins=20, density=True, alpha=0.8, color=mpld.red, label="third component")
pl.hist(trace_b.get_values('logw1') - trace_r.get_values('logw1'), 
        bins=20, density=True, alpha=0.8, color=mpld.blue, label="second component");
pl.ylabel(r'~ probability')
pl.xlabel(r'$\log (\omega_\mathrm{blue}/\omega_\mathrm{red})$')
pl.legend()
pl.savefig('/Users/tgordon/Desktop/prob_omega_br.pdf')

In [ ]:
pl.figure(figsize=(10, 7))
pl.hist(trace_b.get_values('logw2') - trace_g.get_values('logw2'), 
        bins=20, density=True, alpha=0.8, color=mpld.red, label="third component")
pl.hist(trace_b.get_values('logw1') - trace_g.get_values('logw1'), 
        bins=20, density=True, alpha=0.8, color=mpld.blue, label="second component");
pl.ylabel(r'~ probability')
pl.xlabel(r'$\log (\omega_\mathrm{blue}/\omega_\mathrm{green})$')
pl.legend()
pl.savefig('/Users/tgordon/Desktop/prob_omega_bg.pdf')